In [32]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
from cleaner import clean_preliminary,to_categorical_for_cols,remove_nominal_cols
from predictor import train_fill_ml_na,fill_ml_na_col,fill_ml_na
import pandas as pd
import math
import os
import utils
import constants as const
import numpy as np
import pickle
import cleaner as cln

from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

Please uncomment on of these approaches to fill the null values

In [55]:
fill_null_using = "ML"
# fill_null_using = "SIM"

## Reading and Preparing Train Data

The series of functions applied to the training data prepare the data as described in Approach 2 in our report. We briefly describe the use case of each funtion below: <br> 
- *clean_preliminary()*:
    - Handles each columns individually
    - Drops the unwanted rows and columns
- *to_categorical_for_cols()*:
    - Encodes *fuel_type* using dummy variables and *category* using MultiLabelBinarizer()
- *remove_nominal_cols()*:
    - Removes columns that are not needed for further analysis.

In [56]:
df = pd.read_csv('data/train.csv')

In [57]:
df = clean_preliminary(df)
df = to_categorical_for_cols(df)
df = remove_nominal_cols(df)

### Filling Null Values
We have two ways to fill the null values. The training of both these processes take time, further, the saved ML models are very heavy to be pushed on the GitHub repository. We therefore save the data after applying the necessary approach.

#### Machine Learning Based Approach
The function *fill_ml_na()* is used to train Machine learning models, to predict the missing values for each column. In practice, we use RandomizedSearchCV with 200 iterations and 5-fold cross validation. <br>
Since the trained models are heavy, we do not include them in the repository, you can run the below code cell where we have set the number of iterations as 1 and k as 2 for cross validation to allow a quick sample.

In [61]:
filled_train = fill_ml_na(df,training=True,num_iter=1,k_splits=2)

Alternatively, you can use the below code cell to access the training data after filling using the ML models.

In [53]:
filled_train = to_csv('data/Filled_df/filled_df_ML_approach.csv',index=False)

In [26]:
df.isna().sum()

index                         0
manufactured                  0
type_of_vehicle               0
transmission                  0
curb_weight                 310
power                      2045
engine_cap                   50
no_of_owners                  5
depreciation                207
coe                         832
road_tax                   2006
dereg_value                 913
mileage                    3528
omv                           0
arf                          29
price                         0
make_model                    0
car_age                       0
diesel                        0
electric                      0
petrol                        0
petrol-electric               0
missing_category              0
almost_new_car                0
coe_car                       0
consignment_car               0
direct_owner_sale             0
imported_used_vehicle         0
low_mileage_car               0
opc_car                       0
parf_car                      0
premium_

## Train LBGM

In [15]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [17]:
lgb_model = lgb.LGBMRegressor(
#     categorical_feature= [0,2,3,4,8,9,10],
    task = 'predict',
    application = 'regression',
    objective = 'root_mean_squared_error',
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

lgb_model.fit(X_train,y_train) 


preds_lgb_model = lgb_model.predict(X_test)
rmse_lgb = np.sqrt(mean_squared_error(y_test, preds_lgb_model))
print(" RMSE: %f" % (rmse_lgb ))

/Users/kanav7/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
 RMSE: 23583.298923


In [ ]:
Use small max_bin

Use small num_leaves

Use min_data_in_leaf and min_sum_hessian_in_leaf

Use bagging by set bagging_fraction and bagging_freq

Use feature sub-sampling by set feature_fraction

Use bigger training data

Try lambda_l1, lambda_l2 and min_gain_to_split for regularization

Try max_depth to avoid growing deep tree

Try extra_trees

Try increasing path_smooth

In [18]:
parameters = {
    'task' : ['predict'],
    'boosting': ['gbdt' ],
    'objective': ['root_mean_squared_error'],
    'num_iterations': [  1500, 2000,5000  ],
    'learning_rate':[  0.05, 0.005 ],
   'num_leaves':[ 7, 15, 31  ],
   'max_depth' :[ 10,15,25],
   'min_data_in_leaf':[15,25 ],
  'feature_fraction': [ 0.6, 0.8,  0.9],
    'bagging_fraction': [  0.6, 0.8 ],
    'bagging_freq': [   100, 200, 400  ],
     
}


gsearch_lgb = GridSearchCV(lgb_model, param_grid = parameters, n_jobs=6, verbose=10)
gsearch_lgb.fit(X_train,y_train)
 

print('best params')
print (gsearch_lgb.best_params_)
preds_lgb_model = gsearch_lgb.predict(X_test)
rmse_lgb = np.sqrt(mean_squared_error(y_test, preds_lgb_model))
print(" RMSE: %f" % (rmse_lgb ))

Fitting 5 folds for each of 1944 candidates, totalling 9720 fits


KeyboardInterrupt: 

In [111]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

### Train Model RF

In [296]:
df = df_new.copy()
df.dropna(inplace=True)

NameError: name 'df_new' is not defined

In [8]:
X = df.drop(['index','price'],axis=1)
Y = df.price

In [297]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [298]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [259]:
y_train = np.log(y_train)

In [299]:
rf = RandomForestRegressor()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=42, n_jobs = -1)
rf.fit(X_train,y_train)

RandomForestRegressor()

In [300]:
pred = rf.predict(X_test)

In [264]:
pred = np.exp(pred)

In [301]:
mean_squared_error(y_test,pred)**0.5

18660.69487227085

### Test Data

In [14]:
test_data = pd.read_csv('data/test.csv')
test_data = clean_preliminary(test_data,is_test=True)
test_data = to_categorical_for_cols(test_data)
test_data = remove_nominal_cols(test_data)
test_data_new = fill_ml_na(test_data)

/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using vers

make_model has not been trained
omv has not been trained


/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [18]:
test_data.make_model = test_data_new.make_model.fillna(25.0)

In [19]:
test_data_new = test_data_new.fillna(test_data_new.median())

In [20]:
test_data_new['Predict'] = rf_random.predict(test_data_new.drop(['index'],axis=1))

In [19]:
test['Predicted'] = lgb_model.predict(test)

In [20]:
test['id'] = test.index

In [51]:
test.Predicted = test.Predicted.apply(lambda x:round(x / 100.0) * 100.0)

In [52]:
submission = test[['id','Predicted']]

In [53]:
submission.to_csv('data/test_submission_average_weighted.csv',index=False)

In [284]:
test['Predicted'] = np.exp(rf.predict(test))

In [45]:
test = pd.read_csv('data/best_mean.csv')

In [47]:
def weighted_average(x1,x2,x3,x4,x5):
    return 0.3*x1+0.15*x2+0.3*x3+0.15*x4+0.1*x5

In [48]:
test.columns

Index(['id', 'Predicted1', 'Predicted2', 'Predicted3', 'Predicted4',
       'Predicted5', 'Predicted_mean', 'Predicted'],
      dtype='object')

In [49]:
test['Predicted'] = test.apply(lambda x: weighted_average(x.Predicted1, x.Predicted2,x.Predicted3,x.Predicted4,x.Predicted5), axis=1)

In [50]:
test.Predicted

0       158120.0
1       277015.0
2       212895.0
3       163565.0
4        64590.0
          ...   
4995     72135.0
4996    160455.0
4997     55130.0
4998     72980.0
4999     20130.0
Name: Predicted, Length: 5000, dtype: float64

In [33]:
submission = test[['id','Predicted']]

In [34]:
submission

,id,Predicted
0,0,157800
1,1,276000
2,2,214000
3,3,162800
4,4,64500
...,...,...
4995,4995,71700
4996,4996,159200
4997,4997,55300
4998,4998,73100
